<a href="https://colab.research.google.com/github/AndrewSemenets/DSL/blob/main/job2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

```
{'toks': set(token), 'vars': dict(var: definition), 'hvar': var}
token : (class, value)    # токены
class : int
value : str
var : str                 # имя нетерминала
definition : list(rule)
rule : list(var | token)  # правая часть правила
```


1) Удаление посторонних нетерминалов:

---
Заполняем список непосторонних нетерминалов, таких, у которых хотя бы в одном правиле все символы - терминалы или уже известные непосторонние нетерминалы.
После удаляем из грамматики все нетерминалы, которые не вошли в этот список, и правила, в которых они упоминаются.


In [12]:
def remove_external_symbs(grammar):

  toks = grammar['toks']
  vars = grammar['vars']
  non_external_symbs = set() #список непосторонних нетерминалов
  new_nonext_found = True #флаг того, найден ли на предыдущей итерации новый непосторонний нетерминал

  #функция проверяющая, содержит ли правило только токены и непосторонние нетерминалы
  def check_rule(rule) -> bool:
    for rule_part in rule:
      if rule_part in toks or rule_part in non_external_symbs:
        pass
      else:
        return False
    return True

  #ищем непосторонние нетерминалы
  while new_nonext_found:
    new_nonext_found = False
    for non_term, definition in vars.items():
      if non_term not in non_external_symbs:
        for rule in definition:
          if check_rule(rule):
            new_nonext_found = True
            non_external_symbs.add(non_term)
            break
  
  #удаляем из грамматики все упоминания о посторонних нетерминалах (сами нетерминалы и правила с ними)
  new_vars = dict()
  for non_term, definition in vars.items():
    if non_term in non_external_symbs:
      for rule in definition:
        if check_rule(rule):
          if non_term in new_vars.keys():
            new_vars[non_term].append(rule)
          else:
            new_vars[non_term] = list()
            new_vars[non_term].append(rule)

  grammar['vars'] = new_vars

  return grammar


2) Определение исчезающих символов

---
Пусть пустой токен - токен вида (0, 'а')

Находим все нетерминалы, у которых есть хотя бы одно правило, состоящее только из пустых токенов или уже известных исчезающих символов/нетерминалов.



In [13]:
def find_diss_symbs(grammar):
  toks = grammar['toks']
  vars = grammar['vars']
  diss_symbs = set() #список исчезающих символов
  new_diss_symb_found = True #флаг того, найден ли на предыдущей итерации новый исчезающий символ

  #функция проверяющая, содержит ли правило только пустые токены и исчезающие символы
  def check_rule(rule) -> bool:
    for rule_part in rule:
      if rule_part == (0, 'a') or rule_part in diss_symbs:
        pass
      else:
        return False
    return True

  while new_diss_symb_found:
    new_diss_symb_found = False
    for non_term, definition in vars.items():
       if non_term not in diss_symbs:
         for rule in definition:
            if check_rule(rule):
              new_diss_symb_found = True
              diss_symbs.add(non_term)
              break

  return diss_symbs

Пример грамматики и вызов рабочих функций с ней

In [14]:
grammar = {
    'toks': set( [
        (0, 'a'), 
        (1, 'b'), 
        (1, 'c'), 
        (2, 'd')
    ] ),
    'vars': {
        'A' : [['A', (1, 'b')], 
               ['B', 'F'],
               ['C', (0, 'a'), 'D'],
               ['W', (1, 'b')]],
        'W' : [['F', (1, 'c')],
               [(0, 'a')]],
        'B' : [['D', (1, 'c')]],
        'C' : [['B']],
        'D' : [['B', (2, 'd')],
               ['C', (1, 'b')]],
        'F' : [[(2, 'd')],
               [(1, 'b'), 'B'],
               ['W']]
    },
    'hvar': 'F'
}

print("Our grammar without the external non-terminals: ", remove_external_symbs(grammar))

print("Dissaperaing non-terminals: ", find_diss_symbs(grammar))

Our grammar without the external non-terminals:  {'toks': {(2, 'd'), (1, 'b'), (0, 'a'), (1, 'c')}, 'vars': {'A': [['A', (1, 'b')], ['W', (1, 'b')]], 'W': [['F', (1, 'c')], [(0, 'a')]], 'F': [[(2, 'd')], ['W']]}, 'hvar': 'F'}
Dissaperaing non-terminals:  {'F', 'W'}
